### Task 1:
Consider a standard deck of 52 cards.

What is the probability of drawing a red card **bold text**(hearts or diamonds)?

Now, given that you have drawn a red card, what is the probability that the card is a heart?

Given that the drawn card is a face card (Jack, Queen, King), what is the probability that it is a diamond?

Given that the card drawn is a face card, calculate the probability that it is also a spade or a queen.

In [ ]:
suits = ["hearts", "diamonds", "clubs", "spades"]
ranks = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King", "Ace"]

# Create full deck of 52 cards
deck = [(rank, suit) for suit in suits for rank in ranks]
total_cards = len(deck)

# 1. Red cards
red_cards = [card for card in deck if card[1] in ["hearts", "diamonds"]]
p_red = len(red_cards) / total_cards

# 2. Heart cards given red
heart_cards = [card for card in red_cards if card[1] == "hearts"]
p_heart_given_red = len(heart_cards) / len(red_cards)

# 3. Diamond face cards
face_cards = [card for card in deck if card[0] in ['Jack', 'Queen', 'King']]
diamond_faces = [card for card in face_cards if card[1] == "diamonds"]
p_diamond_given_face = len(diamond_faces) / len(face_cards)

# 4. Spade face cards or Queens
spade_faces = [card for card in face_cards if card[1] == "spades"]
queens = [card for card in face_cards if card[0] == "Queen"]
# Combine and remove duplicates
spade_or_queen = set(spade_faces + queens)
p_spade_or_queen_given_face = len(spade_or_queen) / len(face_cards)

# Print results
print(f"Probability of drawing a red card: {p_red:.2f}")
print(f"Probability of heart given red card: {p_heart_given_red:.2f}")
print(f"Probability of diamond given face card: {p_diamond_given_face:.2f}")
print(f"Probability of spade or queen given face card: {p_spade_or_queen_given_face:.2f}")

### Task 2

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. Define structure
model = BayesianNetwork([
    ('Intelligence', 'Grade'),
    ('StudyHours',   'Grade'),
    ('Difficulty',   'Grade'),
    ('Grade',        'Pass')
])

# 2. Define CPTs

# Priors
cpd_I = TabularCPD('Intelligence', 2, [[0.7], [0.3]],
                   state_names={'Intelligence': ['High', 'Low']})

cpd_S = TabularCPD('StudyHours', 2, [[0.6], [0.4]],
                   state_names={'StudyHours': ['Sufficient', 'Insufficient']})

cpd_D = TabularCPD('Difficulty', 2, [[0.4], [0.6]],
                   state_names={'Difficulty': ['Hard', 'Easy']})

# Grade CPT (8 columns for each I×S×D combination)
cpd_G = TabularCPD('Grade', 3,
    # rows: P(G=A), P(G=B), P(G=C)
    [
      [0.90, 0.80, 0.70, 0.60, 0.60, 0.50, 0.40, 0.30],
      [0.08, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.40],
      [0.02, 0.05, 0.10, 0.15, 0.10, 0.15, 0.20, 0.30]
    ],
    evidence=['Intelligence', 'StudyHours', 'Difficulty'],
    evidence_card=[2,2,2],
    state_names={
      'Grade': ['A','B','C'],
      'Intelligence': ['High','Low'],
      'StudyHours': ['Sufficient','Insufficient'],
      'Difficulty': ['Hard','Easy']
    }
)

# Pass CPT
cpd_P = TabularCPD('Pass', 2,
    [[0.95, 0.80, 0.50],   # P(Pass=Yes | G=A,B,C)
     [0.05, 0.20, 0.50]],  # P(Pass=No  | G=A,B,C)
    evidence=['Grade'],
    evidence_card=[3],
    state_names={'Pass':['Yes','No'], 'Grade':['A','B','C']}
)

# 3. Add and check
model.add_cpds(cpd_I, cpd_S, cpd_D, cpd_G, cpd_P)
assert model.check_model()

# 4. Inference
infer = VariableElimination(model)

# (a) P(Pass | StudyHours=Sufficient, Difficulty=Hard)
q1 = infer.query(
    variables=['Pass'],
    evidence={'StudyHours':'Sufficient','Difficulty':'Hard'}
)
print("P(Pass | Sufficient, Hard):")
print(q1)

# (b) P(Intelligence=High | Pass=Yes)
q2 = infer.query(
    variables=['Intelligence'],
    evidence={'Pass':'Yes'}
)
print("\nP(Intelligence=High | Pass=Yes):")
print(q2)


### Task 3

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. Define structure
model = BayesianNetwork([
    ('Disease', 'Fever'),
    ('Disease', 'Cough'),
    ('Disease', 'Fatigue'),
    ('Disease', 'Chills')
])

# 2. Define CPTs

# 2.1 Prior on Disease
cpd_disease = TabularCPD(
    variable='Disease', variable_card=2,
    values=[[0.30],  # Flu
            [0.70]], # Cold
    state_names={'Disease': ['Flu', 'Cold']}
)

# 2.2 Symptom CPTs (Symptom depends only on Disease)
cpd_fever = TabularCPD(
    'Fever', 2,
    [[0.90, 0.50],   # Fever=Yes  (Flu, Cold)
     [0.10, 0.50]],  # Fever=No
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fever': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_cough = TabularCPD(
    'Cough', 2,
    [[0.80, 0.60],   # Cough=Yes
     [0.20, 0.40]],  # Cough=No
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Cough': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_fatigue = TabularCPD(
    'Fatigue', 2,
    [[0.70, 0.30],   # Fatigue=Yes
     [0.30, 0.70]],  # Fatigue=No
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fatigue': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_chills = TabularCPD(
    'Chills', 2,
    [[0.60, 0.40],   # Chills=Yes
     [0.40, 0.60]],  # Chills=No
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Chills': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

# 3. Add CPTs to model
model.add_cpds(
    cpd_disease,
    cpd_fever,
    cpd_cough,
    cpd_fatigue,
    cpd_chills
)
assert model.check_model(), "CPTs are inconsistent!"

# 4. Inference setup
infer = VariableElimination(model)

# --- Task 1: P(Disease | Fever=Yes, Cough=Yes)
q1 = infer.query(
    variables=['Disease'],
    evidence={'Fever': 'Yes', 'Cough': 'Yes'}
)
print("Task 1: P(Disease | Fever=Yes, Cough=Yes)")
print(q1)


# --- Task 2: add Chills=Yes
q2 = infer.query(
    variables=['Disease'],
    evidence={'Fever': 'Yes', 'Cough': 'Yes', 'Chills': 'Yes'}
)
print("\nTask 2: P(Disease | Fever=Yes, Cough=Yes, Chills=Yes)")
print(q2)


# --- Task 3: P(Fatigue=Yes | Disease=Flu)
q3 = infer.query(
    variables=['Fatigue'],
    evidence={'Disease': 'Flu'}
)
print("\nTask 3: P(Fatigue=Yes | Disease=Flu)")
print(q3)


### Task 4
Define a simple Markov Model with three weather states: Sunny, Cloudy, and Rainy.

Create a transition matrix for the weather states, specifying the probabilities of transitioning from one state to another (e.g., from Sunny to Cloudy, or from Cloudy to Rainy).

Simulate the weather for the next 10 days, starting with a Sunny day, using the
Markov Model. Calculate the probability of having at least 3 rainy days over the 10-day period.

In [ ]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([('Spam','ContainsFree'),('Spam','ContainsWinner'),('Spam','ContainsLink'),('SenderReputation','Spam')])

cpd_senderreputation =TabularCPD(variable='SenderReputation',variable_card=2,values=[[0.85],[0.15]],state_names={'SenderReputation':['Good','Bad']})

cpd_spam = TabularCPD(variable='Spam',variable_card=2,values=[[0.2,0.9],[0.8,0.1]],evidence=['SenderReputation'],evidence_card=[2],state_names={'Spam':['Yes','No'],'SenderReputation':['Good','Bad']})

cpd_containsfree = TabularCPD(variable='ContainsFree',variable_card=2,values=[[0.7,0.1],[0.3,0.9]],evidence=['Spam'],evidence_card=[2],state_names={'Spam':['Yes','No'],'ContainsFree':['Yes','No']})

cpd_containswinner = TabularCPD(variable='ContainsWinner',variable_card=2,values=[[0.6,0.05],[0.4,0.95]],evidence=['Spam'],evidence_card=[2],state_names={'Spam':['Yes','No'],'ContainsWinner':['Yes','No']})

cpd_containslink = TabularCPD(variable='ContainsLink',variable_card=2,values=[[0.8,0.3],[0.2,0.7]],evidence=['Spam'],evidence_card=[2],state_names={'Spam':['Yes','No'],'ContainsLink':['Yes','No']})

model.add_cpds(cpd_senderreputation,cpd_spam,cpd_containsfree,cpd_containslink,cpd_containswinner)

assert model.check_model()

infer = VariableElimination(model)

print("\ntask 1 : P(Spam| ContainsFree=Yes,ContainsLink=Yes , SenderReputation = Yes) ")
task1 = infer.query(variables=['Spam'],evidence={'ContainsFree':'Yes','ContainsLink':'Yes','SenderReputation':'Bad'})
print(task1)

print("\n Task2 : P(Spam=No | containsFree = No, ContainsWinner = no, ContainsLink = no, SenderReputation = Good)")
task2 = infer.query(variables=['Spam'],evidence={'ContainsFree':'No','ContainsLink':'No','SenderReputation':'Good'})
print(task2)

model.add_edge('Spam','ContainsUrgent')
cpd_urgent = TabularCPD(variable='ContainsUrgent', variable_card=2,
                        values=[[0.5, 0.9],  # P(Yes | Spam=Yes), P(Yes | Spam=No)
                                [0.5, 0.1]],
                        evidence=['Spam'], evidence_card=[2],state_names={'ContainsUrgent':['Yes','No'],'Spam':['Yes','No']})

model.add_cpds(cpd_urgent)

model.check_model()

infer_updated = VariableElimination(model)

print("\nTask3: P(Spam | ContainsUrgent = Yes)")
task3 = infer_updated.query(variable=['Spam'],evidence={'ContainsUrgent':'Yes'})
print(task3)